<a href="https://colab.research.google.com/github/parks602/Bert_finetuning/blob/main/Kobert_finetune_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LLM Fine Tnning**
영화 리뷰 데이터를 kobert 모델을 통해 긍정/부정 감정 평가를 실시하려한다.
이는 LLM Fine Tunning을 위한 실습이며, 성능 비교를 위해 기존 Kobert 모델의 토크나이저와 기초 딥러닝 모델을 이용한 결과와 비교 실험을 실시한다.


1.   SKT/Kogpt2를 이용한 감정 분류
2.   SKT/Kogpt2와 Fine Tunning을 이용한 감정 분류



# 1. SKT/Kogpt2를 이용한 감정 분류

> 로지스틱 회귀, FNN등을 사용




In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from transformers import PreTrainedTokenizerFast
import warnings
warnings.filterwarnings('ignore')

#토크나이저 불러오기
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

#모델 불러오기
model = TFBertModel.from_pretrained("bert-base-multilingual-cased", output_hidden_states = True)


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [2]:
def encode(sents, tokenizer):
    input_ids = []
    attention_mask = []

    for text in sents:
        tokenized_text = tokenizer.encode_plus(text,
                                            max_length=20,
                                            add_special_tokens = True,
                                            pad_to_max_length=True,
                                            return_attention_mask=True)

        input_ids.append(tokenized_text['input_ids'])
        attention_mask.append(tokenized_text['attention_mask'])
    return tf.convert_to_tensor(input_ids, dtype=tf.int32), tf.convert_to_tensor(attention_mask, dtype=tf.int32)

In [3]:
#코랩 경로 이동
from google.colab import drive
drive.mount('/content/drive')

#데이터 불러오기
with open('/content/drive/MyDrive/Colab Notebooks/Sample_data/ratings_test.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f ]
    docs = [(doc[1], int(doc[2])) for doc in docs[1:]]
    texts, labels = zip(*docs)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
len(texts)

50000

In [5]:
#각 영화 리뷰의 토큰ID와 attention mask 정보 추출
tokenized_sents = encode(texts[:3000], tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [6]:
for layer in model.layers:
    layer.trainable=False
#테스트 용으로 파라미터 고정

In [7]:
outputs = model(tokenized_sents[0], attention_mask = tokenized_sents[1])

outputs[2][-1].shape

TensorShape([3000, 20, 768])

3000=리뷰의 수 / 20=각 리뷰를 구성하는 토큰 수 / 768=각 토큰을 구성하는 벡터 차원

In [13]:
#독립변수 지정
hidden_states = outputs[2][-1]
features = hidden_states[:,0,:].numpy()

#종속변수 지정
selected_labels=labels[:3000]

In [14]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, selected_labels, test_size=0.2, random_state=0)

from sklearn.linear_model import LogisticRegression
lr2 = LogisticRegression(C=0.1, penalty='l2', solver='saga', max_iter=1000)
lr2.fit(train_features, train_labels)
pred_labels = lr2.predict(test_features)

#정확도 확인
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, pred_labels)

0.62

In [ ]:
#원핫 인코딩
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(selected_labels)

#train/test 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y_one_hot, test_size=0.2)

#FNN 모델 정의
from tensorflow.keras import models
model = models.Sequential()

from tensorflow.keras import layers
model.add(layers.Dense(64, activation = 'tanh', input_shape=(X_train.shape[1],)))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation = 'tanh'))
model.add(layers.Dense(2, activation='softmax'))

#모델 학습
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

#테스트셋 성과 평가
model.evaluate(X_test,y_test)[1]

# 2.  SKT/Kogpt2와 Fine Tunning을 이용한 감정 분류


In [5]:
#데이터 불러오기
with open('/content/drive/MyDrive/Colab Notebooks/Sample_data/ratings_train.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f ]
    docs = [(doc[1], int(doc[2])) for doc in docs[1:]]
    texts, labels = zip(*docs)

print(len)

<built-in function len>


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, random_state=42, test_size=0.2)

In [7]:
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

def encode(data, tokenizer):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    for text in data:
        tokenized_text = tokenizer.encode_plus(text,
                                            max_length=50,
                                            add_special_tokens = True,
                                            pad_to_max_length=True,
                                            return_attention_mask=True,
                                              truncation=True)
        input_ids.append(tokenized_text['input_ids'])
        attention_masks.append(tokenized_text['attention_mask'])
        token_type_ids.append(tokenized_text['token_type_ids'])

    return input_ids, attention_masks, token_type_ids

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


fine tuning시 token_type_ids 필요

In [8]:
#학습 데이터
train_input_ids, train_attention_masks, train_token_type_ids = encode(X_train, tokenizer)

#테스트 데이터
test_input_ids, test_attention_masks, test_token_type_ids = encode(X_test, tokenizer)

In [10]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
      }, label


def data_encode(input_ids_list, attention_mask_list, token_type_ids_list, label_list):
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

BATCH_SIZE=32

train_data_encoded = data_encode(train_input_ids, train_attention_masks, train_token_type_ids,y_train).shuffle(10000).batch(BATCH_SIZE)
test_data_encoded = data_encode(test_input_ids, test_attention_masks, test_token_type_ids, y_test).batch(BATCH_SIZE)

In [ ]:

model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2
)


optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

NUM_EPOCHS = 5
history = model.fit(train_data_encoded, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=test_data_encoded)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1764/3750 [=============>................] - ETA: 11:11 - loss: 0.5673 - accuracy: 0.6903

In [ ]:
# loss 확인
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()


# accuracy 확인
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'])
plt.show()